In [1]:
# you also need to start your local solr server
# cd ~/your solr installation path/solr-9.0.0-SNAPSHOT/ && ./bin/solr start

import sys,os,time,re,csv,string
import pysolr
import pandas as pd
import numpy as np
import math
import nltk
import torch
from tqdm import tqdm
from ast import literal_eval
from datetime import datetime
import nltk
import torch
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
sys.path.append('../Evaluation/scripts/')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from solrclient import *
from cal_features import *
from runeval import runeval
from runs_export import *

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We are using:  cpu


In [6]:
#read topic file
topicfilepath = '../Data/Topic/trec2021/topicquery.csv'
queries = pd.read_csv(topicfilepath)
queries = queries.rename(columns={'num': 'qnum'})
queries.head(1)

,qnum,docid,url,title,desc,narr,subtopics,author,published_date,qcontent,Query,Key_Words
0,Number: 936,f831cae6-bfa4-11e1-9ce8-ff26651238d0,https://www.washingtonpost.com/local/obituarie...,Nora Ephron dies at 71,I'm looking for information on the passing of ...,Please provide information as to when author a...,"[""Find details of Nora Ephron's life and accom...",Adam Bernstein,1.340756e+12,"ObituariesNora Ephron, prolific author and scr...","{'when did she write heartburn', 'why did nora...","{'screenwriter', 'earliest years', 'new york',..."


In [8]:
def retrieval(resftp,queries,composition):
    
    # composition = 1, desc+title+kw+t5
    # composition = 2, desc+title+t5
    # composition = 3, desc+title+kw
    # composition = 4, desc+title
    # composition = 5, desc
    
    #search with Desc without the field kw,pdqueries
    rows = 200
    for i in range(len(queries)):
        client = SolrClient(server_port='8983', server_core='WapoIRT5')
        query = queries.loc[i,:]
        
        kwlist = getpredictquery(query['Key_Words'])
        querylist = getpredictquery(query['Query'])
        
        if composition == 1:
            querytext = query['desc']+query['title']+turnstring(kwlist)+turnstring(querylist)
        elif composition == 2:
            querytext = query['desc']+query['title']+turnstring(querylist)
        elif composition == 3:    
            querytext = query['desc']+query['title']+turnstring(kwlist)
        elif composition == 4:
            querytext = query['desc']+query['title']
        elif composition == 5:
            querytext = query['desc']
        else:
            print("Wrong Number. Please enter int number from 1-5.")
            return
        
        client.get_results(query,querytext,rows)
        client.export_results(resftp + query['qnum'][-3:]+'.csv',rows)
    print("Retrieval successed!")

In [11]:
#test
resultpath = './result/IRexpanded_descTitleKey/'
retrieval(resftp=resultpath, queries=queries, composition=3)

Retrieval successed!


In [ ]:
#rerank

def semantic_rerank(resftp,featurepath,outputpath,rerankmethod=rerank_Freshness):
    
    #rerankmethod = rerank_Freshness/rerank_Title/rerank_Title_Fr
    topicfilepath = '../Data/Topic/trec2021/clsquery.csv'
    queries = pd.read_csv(topicfilepath)
    files = os.listdir(resftp)
    try:
        files.remove('.ipynb_checkpoints')
    except:
        pass
    
    print("Calculating semantic features on %s files"%len(files))
    for file in tqdm(files):
        temp = pd.read_csv(resftp+file)
        features_df = cal_featues(df_queries=queries,df_result=temp)
        features_df.to_csv(featurepath+file,index=False)
    print("Re-ranking...")
    for file in tqdm(files):
        temp = pd.read_csv(featurepath+file)
        features_df = rerankmethod(temp)
        features_df.to_csv(outputpath+file,index=False)
    
    print("Re-ranking Done!")

featurepath = './result/features/'
outputpath = './result/reranked/'
semantic_rerank(resultpath,featurepath,outputpath,rerankmethod=rerank_Freshness)

  0%|          | 0/51 [00:00<?, ?it/s]

Calculating semantic features on 51 files


In [3]:
# evaluation
def evaluate(resfoldername,details=False):
    resftp = './result/%s/'%(resfoldername)
    #resftp = '/home/jovyan/playground/projects/TREC2021/NewsTrack/BaseLine/%s/'%(resfolder)
    ExportLtrRun(resftp,resfoldername)
    #runpath = '/home/jovyan/playground/projects/TREC2021/NewsTrack/BaseLine/%s.csv.gz'%(resfoldername)
    runpath = '/home/jovyan/playground/projects/TREC2021/NewTrack_Github/Pipeline/result/'+ '%s.csv.gz'%(resfoldername)
    evalpath = '/home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/res_topics/%s.trec_eval'%(resfoldername)
    if details:
        scores = runeval(runpath,evalpath,detail=True)
        return scores
    else:
        runeval(runpath,evalpath)

# evaluate("reranked")
scores = evaluate("reranked",details=True)
scores

delete old reranked.csv.gz
Saving top 100 rank of 51 topics to one file...
Total length is : 5100
quick run result look:
    qnum queryid                                    id  rank  FinalScore  \
0   936      Q0  fa0c8e88-c12e-11e1-b42e-930aab678615     2    1.313619   

     runtag  
0  reranked  
Doing Evaluation ...
cd /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/scripts && python eval-background.py /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Pipeline/result/reranked.csv.gz /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/res_topics/reranked.trec_eval
trec.eval file has been save to /home/jovyan/playground/projects/TREC2021/NewTrack_Github/Evaluation/res_topics/reranked.trec_eval


[0.8254,
 0.7183,
 0.1566,
 0.2816,
 0.2828,
 0.5701,
 0.3314,
 0.7238,
 0.0515,
 0.2673,
 0.2533,
 0.7555,
 0.1675,
 0.2778,
 0.4038,
 0.5548,
 0.8728,
 0.371,
 0.1677,
 0.893,
 1.0,
 0.3683,
 0.3757,
 0.3325,
 0.5363,
 0.2389,
 0.2117,
 0.5545,
 0.9344,
 0.0,
 0.2865,
 0.1712,
 0.8688,
 0.8516,
 0.6439,
 0.3024,
 0.786,
 0.3314,
 0.647,
 0.8778,
 0.2533,
 0.5531,
 0.463,
 0.0333,
 0.7925,
 0.784,
 0.7544,
 0.475,
 0.8337,
 0.3765,
 0.2072,
 0.4857]